# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 03**: Data Cleaning and Transformation Pipeline

**Date**: September 18th 2025

**Student Name**:Jaime Antonio Contreras Barragan 

**Professor**: Pablo Camarillo Ramirez

In [2]:
import findspark
findspark.init()

# Create SparkSession

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on data sources (Files)") \
    .master("spark://75bb008a135c:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/21 22:20:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Define schema

In [7]:
from jaimodule.spark_utils import SparkUtils
airlines_schema_columns = [("index", "int"), 
     ("airline", "string"), 
     ("flight", "string"),
     ("source_city", "string"),
     ("departure_time", "string"),
     ("stops", "string"),
     ("arrival_time", "string"),
     ("destination_city", "string"),
     ("class", "string"),
     ("duration", "float"),
     ("days_left", "int"),
     ("price", "int")
     ]
airlines_schema = SparkUtils.generate_schema(airlines_schema_columns)
airlines_schema 

StructType([StructField('index', IntegerType(), True), StructField('airline', StringType(), True), StructField('flight', StringType(), True), StructField('source_city', StringType(), True), StructField('departure_time', StringType(), True), StructField('stops', StringType(), True), StructField('arrival_time', StringType(), True), StructField('destination_city', StringType(), True), StructField('class', StringType(), True), StructField('duration', FloatType(), True), StructField('days_left', IntegerType(), True), StructField('price', IntegerType(), True)])

# Load CSV

In [39]:
df_airlines = spark.read \
                .option("header", "true") \
                .schema(airlines_schema) \
                .csv("/opt/spark/work-dir/data/airline/")


# Data Cleaning

In [40]:
from pyspark.sql.functions import trim, col, count, isnull, when
print(f"number of records before cleaning: {df_airlines.count()}")
# Get number of null values for each column before cleaning 
df_airlines.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

# Perform data cleaning with trim (column by column)
airlines_clean = df_airlines \
        .dropDuplicates(["index"]) \
        .withColumn("airline", trim("airline")) \
        .withColumn("source_city", trim("source_city")) \
        .withColumn("destination_city", trim("destination_city")) \
        .filter(col("price").isNotNull())

# Simply using dropna()
airlines_clean_v2 = df_airlines.dropna()

print(f"number of records after cleaning with trim: {airlines_clean.count()}")
airlines_clean.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

print(f"number of records after cleaning with dropna: {airlines_clean_v2.count()}")
airlines_clean_v2.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

number of records before cleaning: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



number of records after cleaning with trim: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



number of records after cleaning with dropna: 300153


[Stage 71:=============================>                            (1 + 1) / 2]

+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



# MAPING STOPS

In [41]:
from pyspark.sql.functions import when, lit
# Perform the 1st transformation
airlines_t1 = airlines_clean_v2.withColumn("stops",
                                           when(airlines_clean_v2.stops == "zero", lit(0))
                                           .when(airlines_clean_v2.stops == "one", lit(1))
                                           .when(airlines_clean_v2.stops == "two_or_more", lit(2))
                                          ) 
airlines_t1.show(n=5)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening|    0|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning|    0|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning|    0|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|    3| Vistara| UK-995|      Delhi|       Morning|    0|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    4| Vistara| UK-963|      Delhi|       Morning|    0|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+-----+--------+-------+

# New Column Route

In [42]:
from pyspark.sql.functions import concat_ws

airlines_clean_v3 = airlines_t1.withColumn("route",
                                           concat_ws("→",col("source_city"),col("destination_city")))
airlines_clean_v3.show(n=5)

                                

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+------------+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|       route|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+------------+
|    0|SpiceJet|SG-8709|      Delhi|       Evening|    0|        Night|          Mumbai|Economy|    2.17|        1| 5953|Delhi→Mumbai|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning|    0|      Morning|          Mumbai|Economy|    2.33|        1| 5953|Delhi→Mumbai|
|    2| AirAsia| I5-764|      Delhi| Early_Morning|    0|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|Delhi→Mumbai|
|    3| Vistara| UK-995|      Delhi|       Morning|    0|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|Delhi→Mumbai|
|    4| Vistara| UK-963|      Delhi|       Morning|    

# Departure and Arrival to numerical

In [43]:
airlines_clean_v3.select("departure_time").distinct().show()
airlines_clean_v3.select("arrival_time").distinct().show()

+--------------+
|departure_time|
+--------------+
|       Evening|
|       Morning|
|    Late_Night|
|     Afternoon|
| Early_Morning|
|         Night|
+--------------+



[Stage 79:=============================>                            (1 + 1) / 2]

+-------------+
| arrival_time|
+-------------+
|      Evening|
|      Morning|
|   Late_Night|
|    Afternoon|
|Early_Morning|
|        Night|
+-------------+



In [44]:

airlines_t1 = airlines_clean_v3 .withColumn("departure_time",
                                            when(airlines_clean_v3.departure_time == "Early_Morning", lit(0)).when(airlines_clean_v3.departure_time == "Morning", lit(1))
                                            .when(airlines_clean_v3.departure_time == "Afternoon", lit(2)).when(airlines_clean_v3.departure_time == "Evening", lit(3))
                                            .when(airlines_clean_v3.departure_time == "Night", lit(4)).when(airlines_clean_v3.departure_time == "Late_Night", lit(5))
) 
airlines_t1.show(n=5)
airlines_t1 = airlines_t1 .withColumn("arrival_time",
                                            when(airlines_t1.arrival_time == "Early_Morning", lit(0)).when(airlines_t1.arrival_time == "Morning", lit(1))
                                            .when(airlines_t1.arrival_time == "Afternoon", lit(2)).when(airlines_t1.arrival_time == "Evening", lit(3))
                                            .when(airlines_t1.arrival_time == "Night", lit(4)).when(airlines_t1.arrival_time == "Late_Night", lit(5))
) 
airlines_t1.show(n=5)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+------------+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|       route|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+------------+
|    0|SpiceJet|SG-8709|      Delhi|             3|    0|        Night|          Mumbai|Economy|    2.17|        1| 5953|Delhi→Mumbai|
|    1|SpiceJet|SG-8157|      Delhi|             0|    0|      Morning|          Mumbai|Economy|    2.33|        1| 5953|Delhi→Mumbai|
|    2| AirAsia| I5-764|      Delhi|             0|    0|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|Delhi→Mumbai|
|    3| Vistara| UK-995|      Delhi|             1|    0|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|Delhi→Mumbai|
|    4| Vistara| UK-963|      Delhi|             1|    

# Expensive column

In [45]:
airlines_t1 = airlines_t1 .withColumn("is_expensive",when(col("price")>6000,True).otherwise(False))
airlines_t1.show(n=5)

+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+------------+------------+
|index| airline| flight|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|price|       route|is_expensive|
+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+------------+------------+
|    0|SpiceJet|SG-8709|      Delhi|             3|    0|           4|          Mumbai|Economy|    2.17|        1| 5953|Delhi→Mumbai|       false|
|    1|SpiceJet|SG-8157|      Delhi|             0|    0|           1|          Mumbai|Economy|    2.33|        1| 5953|Delhi→Mumbai|       false|
|    2| AirAsia| I5-764|      Delhi|             0|    0|           0|          Mumbai|Economy|    2.17|        1| 5956|Delhi→Mumbai|       false|
|    3| Vistara| UK-995|      Delhi|             1|    0|           2|          Mumbai|Economy|    2.25|        1| 595

# Get the average price per airline.

In [48]:
from pyspark.sql.functions import avg

airlines_t1.groupBy("airline").agg(avg("price").alias("avg_price")).show()

[Stage 88:=============================>                            (1 + 1) / 2]

+---------+------------------+
|  airline|         avg_price|
+---------+------------------+
|   Indigo| 5324.216303339517|
| SpiceJet| 6179.278881367218|
|Air_India| 23507.01911190229|
|  AirAsia|4091.0727419555224|
| GO_FIRST| 5652.007595045959|
|  Vistara| 30396.53630170735|
+---------+------------------+



# Average duration per route.



In [49]:
airlines_t1.groupBy("route").agg(avg("duration").alias("avg_duration")).show()

[Stage 91:=============================>                            (1 + 1) / 2]

+-------------------+------------------+
|              route|      avg_duration|
+-------------------+------------------+
|      Kolkata→Delhi| 11.60498857561711|
|     Kolkata→Mumbai|12.991932481150478|
|     Mumbai→Chennai|12.665900287564627|
|    Kolkata→Chennai|14.774181563782903|
|   Hyderabad→Mumbai|11.962923295795918|
|    Delhi→Hyderabad|12.518350118710492|
|      Delhi→Kolkata| 12.73596614766045|
|  Bangalore→Kolkata|13.099143404859825|
|       Mumbai→Delhi|  9.81805726844943|
|  Kolkata→Bangalore| 13.79294687524098|
|   Bangalore→Mumbai| 10.90507225639642|
|  Bangalore→Chennai|14.480207509137166|
|       Delhi→Mumbai|10.367774213738123|
|      Delhi→Chennai|12.433964745763944|
|  Hyderabad→Chennai|13.293238468912078|
|   Mumbai→Bangalore|11.612022516178817|
|    Bangalore→Delhi|  9.77995566082195|
|    Hyderabad→Delhi|10.829816602522587|
|Hyderabad→Bangalore| 12.09331678643705|
|     Mumbai→Kolkata|12.836848115489666|
+-------------------+------------------+
only showing top

# Minimum and maximum price per airline.

In [52]:
from pyspark.sql.functions import min, max

airlines_t1.groupBy("airline").agg(min("price").alias("min"),max("price").alias("max")).show()

[Stage 97:=============================>                            (1 + 1) / 2]

+---------+----+------+
|  airline| min|   max|
+---------+----+------+
|   Indigo|1105| 31952|
| SpiceJet|1106| 34158|
|Air_India|1526| 90970|
|  AirAsia|1105| 31917|
| GO_FIRST|1105| 32803|
|  Vistara|1714|123071|
+---------+----+------+



# Count flights by departure_time category.

In [54]:
airlines_t1.groupBy("departure_time").count().orderBy("departure_time").show()

[Stage 103:============================>                            (1 + 1) / 2]

+--------------+-----+
|departure_time|count|
+--------------+-----+
|             0|66790|
|             1|71146|
|             2|47794|
|             3|65102|
|             4|48015|
|             5| 1306|
+--------------+-----+

